In [1]:
import sqlite3
from transformers import pipeline
import re
import json

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng full nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS full (field TEXT, content TEXT)''')
conn.commit()

# Hàm xóa dữ liệu cũ trong bảng full
def clear_old_data():
    cursor.execute('DELETE FROM full')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng full.")

# Hàm làm sạch văn bản bằng Regular Expressions
def clean_text(text):
    # Loại bỏ các ký tự không mong muốn
    cleaned_text = re.sub(r'[\"_;(-]', '', text)  # Loại bỏ dấu ngoặc kép, gạch dưới và dấu chấm phẩy
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Thay nhiều khoảng trắng thành một
    return cleaned_text

# Hàm chỉnh sửa chính tả và lưu vào DB
def correct_spelling_and_save():
    # Khởi tạo pipeline cho việc chỉnh sửa chính tả
    corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction-v2")

    # Lấy dữ liệu từ bảng detailtrc
    cursor.execute('SELECT input_name, output_path FROM detailtrc')
    detailtrc_rows = cursor.fetchall()

    # Lấy dữ liệu từ bảng detailsau
    cursor.execute('SELECT input_name, output_path FROM detailsau')
    detailsau_rows = cursor.fetchall()

    # Lưu kết quả vào bảng full
    for input_name, output_path in detailtrc_rows + detailsau_rows:  # Kết hợp cả hai bảng
        corrected_text = corrector(output_path)[0]['generated_text']
        cleaned_text = clean_text(corrected_text)  # Làm sạch văn bản đã chỉnh sửa
        cursor.execute('INSERT INTO full (field, content) VALUES (?, ?)', (input_name, cleaned_text))

    conn.commit()
    print("Dữ liệu đã được lưu vào bảng full.")

# Hàm xuất dữ liệu ra file JSON theo định dạng yêu cầu
def export_to_json():
    cursor.execute('SELECT field, content FROM full')
    rows = cursor.fetchall()

    # Tạo một từ điển để lưu trữ kết quả theo định dạng yêu cầu
    json_output = {}

    for field, content in rows:
        # Cập nhật giá trị cho từng trường trong json_output
        if field in json_output:
            # Nếu trường đã tồn tại, có thể gộp hoặc xử lý theo cách nào đó nếu cần thiết.
            json_output[field] += f", {content}"  # Gộp nội dung nếu cần thiết (có thể điều chỉnh)
        else:
            json_output[field] = content

    # Xuất ra file JSON
    with open('../data/output.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_output, json_file, ensure_ascii=False, indent=4)
    
    print("Dữ liệu đã được xuất ra file output.json.")

# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Chạy hàm chỉnh sửa chính tả và lưu kết quả
correct_spelling_and_save()

# Xuất dữ liệu ra file JSON
export_to_json()

# Đóng kết nối cơ sở dữ liệu
conn.close()


Đã xóa dữ liệu cũ trong bảng full.


Device set to use cpu


Dữ liệu đã được lưu vào bảng full.
Dữ liệu đã được xuất ra file output.json.
